In [10]:
from math import cos, asin, sqrt, pi
import json
import deap

In [11]:
import random
import numpy as np
from deap import base, creator, tools, algorithms
from joblib import Parallel, delayed


def _distance(location1, location2):
    lat1, lon1 = location1
    lat2, lon2 = location2
    p = pi/180
    a = 0.5 - cos((lat2-lat1)*p)/2 + cos(lat1*p) * cos(lat2*p) * (1-cos((lon2-lon1)*p))/2
    return 12742 * asin(sqrt(a)) #2*R*asin...


def _initIndividual(pcls, lengths, total):
    part = pcls(np.random.choice(total, lengths, replace=False))
    return part


def _mutIndividual(individual, up, indpb):
    for i in range(len(individual)):
        rn = random.random()
        if rn < indpb:
            nb = random.randint(0, up-1)
            if nb in individual:
                index = individual.index(nb)
                value = individual[index]
                individual[index] = individual[i]
                individual[i] = value
            else:
                individual[i] = nb
    return individual,


def _cxIndividual(ind1, ind2):
    size = len(ind1)
    start = random.randint(1, size-2)
    end = random.randint(start+1, size-1)
    children1 = [None] * start + ind1[start:end] + [None] * (size - end)
    children2 = [None] * start + ind2[start:end] + [None] * (size - end)
    to_fill = [_ for _ in range(end, size)] + [_ for _ in range(0, start)]
    ind1_order = ind1[end:] + ind1[:end]
    ind2_order = ind2[end:] + ind2[:end]
    for i in to_fill:
        for j in ind1_order:
            if j not in children2:
                children2[i] = j
                break
        for j in ind2_order:
            if j not in children1:
                children1[i] = j
                break
    for i in range(size):
        ind1[i] = children1[i]
        ind2[i] = children2[i]
    return ind1, ind2


class EvoTravel():
    def __init__(self, locations, n_population=100, n_generations=10, cxpb=0.5, mutpb=0.05, tournament_size=3, hof_size=1, verbose=False, n_jobs=-1, jobs_verbose=0, **kwargs):
        self._locations_info = locations
        self._locations = [(float(location.get('lat')), float(location.get('lng'))) for location in locations]
        self.n_population = n_population
        self.n_generations = n_generations
        self.cxpb = cxpb
        self.mutpb = mutpb
        self.tournament_size = tournament_size
        self.verbose = verbose
        self.n_jobs = n_jobs
        self.jobs_verbose = jobs_verbose
        self.hof_ = None
        self.hof_size = hof_size
        self.stats_ = None
        self.hist_ = None
        self.logbook_ = None
        self.pop_ = None
        self.generations = []

    def _nanmean(self, individuals):
        return np.nanmean([fitness for _, fitness in individuals])

    def _nanmin(self, individuals):
        return np.nanmin([fitness for _, fitness in individuals])

    def _nanmax(self, individuals):
        max_individual = max(individuals, key=lambda item: item[1])
        print('Best in gen with fitness {}: {}'.format(max_individual[1], max_individual[0]))
        self.generations.append(max_individual[0])
        return np.nanmax([fitness for _, fitness in individuals])

    def _nanstd(self, individuals):
        return np.nanstd([fitness for _, fitness in individuals])
        
    def _evaluate(self, individual, origin, destination):
        ind = list(individual)
        locations = list([origin]) + [self._locations[ind[i]] for i in range(0, len(ind))] + list([destination])
        return -sum([_distance(locations[i-1], locations[i]) for i in range(1, len(locations))]),

    def _parallel_map(self, f, *iters):
        return Parallel(n_jobs=self.n_jobs, verbose=self.jobs_verbose)(delayed(f)(list(*args)) for args in zip(*iters))

    def fit(self, travels, origin, destination):
        total = len(self._locations)
        
        creator.create("FitnessMax", base.Fitness, weights=(1.0,))
        creator.create("Individual", list, fitness=creator.FitnessMax)

        toolbox = base.Toolbox()

        # Structure initializers
        toolbox.register("individual", _initIndividual, creator.Individual, lengths=travels, total=total)

        # define the population to be a list of individuals
        toolbox.register("population", tools.initRepeat, list, toolbox.individual)

        # map parallel
        #toolbox.register("map", self._parallel_map)

        #----------
        # Operator registration
        #----------
        # register the goal / fitness function
        toolbox.register("evaluate", self._evaluate, origin=origin, destination=destination)

        # register the crossover operator
        toolbox.register("mate", _cxIndividual)

        # register a mutation operator with a probability
        toolbox.register("mutate", _mutIndividual, indpb=self.mutpb, up=total)

        # operator for selecting individuals for breeding the next
        # generation: each individual of the current generation
        # is replaced by the 'fittest' (best) of three individuals
        # drawn randomly from the current generation.
        toolbox.register("select", tools.selTournament, tournsize=self.tournament_size)

        pop = toolbox.population(n=self.n_population)
        hof = tools.HallOfFame(self.hof_size)

        # Stats
        stats = tools.Statistics(lambda ind: (ind, ind.fitness.values))
        stats.register("avg", self._nanmean)
        stats.register("min", self._nanmin)
        stats.register("max", self._nanmax)
        stats.register("std", self._nanstd)

        # History
        hist = tools.History()
        toolbox.decorate("mate", hist.decorator)
        toolbox.decorate("mutate", hist.decorator)
        hist.update(pop)

        pop, logbook = algorithms.eaSimple(pop, toolbox, cxpb=self.cxpb, mutpb=self.mutpb,
                                                   ngen=self.n_generations, stats=stats,
                                                   halloffame=hof, verbose=self.verbose)

        if self.verbose:
            print("Best individual has fitness: %s and params: %s" % (hof[0].fitness.values, hof[0]))
            
        self.best = hof[0]
        self.hof_ = hof
        self.stats_ = stats
        self.hist_ = hist
        self.logbook_ = logbook
        self.pop_ = pop

In [12]:
with open('mexico.json') as json_file:
    data = json.load(json_file)

In [13]:
origin = [20.6538687, -87.1067159]

In [14]:
data = data[:3] + data[4:]

In [15]:
data

[{'city': 'Ciudad de Mexico', 'lat': '19.390519', 'lng': '-99.4238064'},
 {'city': 'Cancun', 'lat': '21.1214886', 'lng': '-86.9192733'},
 {'city': 'Chichén-Itzá', 'lat': '20.6787867', 'lng': '-88.5706656'},
 {'city': 'Cabo san lucas', 'lat': '22.8962649', 'lng': '-109.9505077'},
 {'city': 'Tulum', 'lat': '19.7242834', 'lng': '-87.6788368'},
 {'city': 'Puerto vallarta', 'lat': '20.6409485', 'lng': '-105.2595437'},
 {'city': 'Cozumel', 'lat': '20.2203773', 'lng': '-87.8405077'},
 {'city': 'isla mujeres', 'lat': '21.2439809', 'lng': '-86.798255'},
 {'city': 'Mérida', 'lat': '20.9802115', 'lng': '-89.7029582'},
 {'city': 'Oaxaca', 'lat': '17.0813771', 'lng': '-96.770751'},
 {'city': 'Teotihuacan', 'lat': '19.6881128', 'lng': '-98.8846183'},
 {'city': 'Acapulco', 'lat': '16.8356128', 'lng': '-99.9323486'},
 {'city': 'San miguel de allende', 'lat': '20.9150924', 'lng': '-100.762212'},
 {'city': 'Guanajuato', 'lat': '21.025147', 'lng': '-101.2753897'},
 {'city': 'Guadalajara', 'lat': '20.6738

In [16]:
travel = EvoTravel(data[1:], n_population=50000, n_generations=1000, mutpb=0.4, verbose=True)

In [17]:
travel.fit(travels=46, origin=origin, destination=origin)

/Users/rodrigoazevedosantos/Library/Caches/pypoetry/virtualenvs/evotravel-NPHoPNJS-py3.11/lib/python3.11/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/Users/rodrigoazevedosantos/Library/Caches/pypoetry/virtualenvs/evotravel-NPHoPNJS-py3.11/lib/python3.11/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


Best in gen with fitness (-35680.91000940275,): [9, 24, 23, 42, 16, 27, 30, 5, 25, 21, 26, 2, 34, 45, 18, 40, 0, 33, 13, 31, 22, 8, 38, 37, 44, 14, 35, 6, 17, 20, 12, 41, 39, 7, 4, 10, 32, 29, 43, 36, 19, 15, 11, 1, 28, 3]
gen	nevals	avg     	min     	max     	std    
0  	50000 	-56472.8	-73721.6	-35680.9	4510.65
Best in gen with fitness (-36517.05418628475,): [45, 27, 26, 37, 42, 24, 23, 33, 6, 41, 34, 44, 9, 12, 36, 31, 13, 38, 2, 28, 4, 35, 16, 19, 3, 20, 32, 0, 1, 25, 14, 17, 21, 40, 43, 11, 10, 22, 39, 8, 29, 5, 15, 18, 7, 30]
1  	34865 	-54155.8	-72039.9	-36517.1	4348.01
Best in gen with fitness (-36163.39505435941,): [1, 45, 42, 40, 10, 29, 41, 16, 0, 21, 34, 24, 38, 26, 4, 37, 44, 11, 31, 2, 13, 23, 9, 22, 20, 3, 25, 19, 18, 28, 12, 36, 8, 5, 14, 43, 7, 15, 6, 27, 30, 17, 39, 33, 35, 32]
2  	34879 	-52859.3	-75264.3	-36163.4	4639.98
Best in gen with fitness (-33462.13396203729,): [45, 26, 12, 32, 10, 2, 31, 43, 9, 6, 15, 37, 41, 33, 1, 44, 40, 13, 11, 36, 30, 7, 18, 14, 19, 29,

KeyboardInterrupt: 

In [18]:
for i in travel.best:
    print(str(travel._locations_info[i]) + '\n')

AttributeError: 'EvoTravel' object has no attribute 'best'

In [17]:
travels = travel.generations + [travel.best]

In [18]:
origin

[20.6538687, -87.1067159]

In [19]:
import folium

trvl = travels[-1]
sample_coords = [origin] + [[float(travel._locations_info[i]['lat']), float(travel._locations_info[i]['lng'])] for i in trvl] + [origin]

# Build map 
map_nyc = folium.Map(location=(19.3910038,-99.2836969), zoom_start=3, 
tiles='cartodbpositron', width=270, height=480)

# Plot coordinates using comprehension list
[folium.CircleMarker(sample_coords[i], radius=1,
                color='#0080bb', fill_color='#0080bb').add_to(map_nyc) 
for i in range(len(sample_coords))]

#add lines
folium.PolyLine(sample_coords, color="#0080bb", weight=2.5, opacity=1).add_to(map_nyc)

In [20]:
map_nyc

In [142]:
import folium
import datetime as dt
import random as rnd

import os
import time
from selenium import webdriver

for idx in range(0, len(travels), 10):
    trvl = travels[idx]
    sample_coords = [origin] + [[float(travel._locations_info[i]['lat']), float(travel._locations_info[i]['lng'])] for i in trvl] + [origin]

    # Build map 
    map_nyc = folium.Map(location=(19.3910038,-99.2836969), zoom_start=3, 
    tiles='cartodbpositron', width=270, height=480)

    # Plot coordinates using comprehension list
    [folium.CircleMarker(sample_coords[i], radius=1,
                    color='#0080bb', fill_color='#0080bb').add_to(map_nyc) 
    for i in range(len(sample_coords))]

    #add lines
    folium.PolyLine(sample_coords, color="#0080bb", weight=2.5, opacity=1).add_to(map_nyc)
    
    # save html and png
    path = 'C:/Users/cs317917/Documents/GitHub/evolutionary_traveling_salesman'
    os.chdir(path)

    delay=5
    map_nyc.save('testmap.html')

    fn='testmap.html'
    tmpurl='file:///{path}/{mapfile}'.format(path=os.getcwd(), mapfile=fn)

    browser = webdriver.Chrome()
    browser.get(tmpurl)
    #Give the map tiles some time to load
    time.sleep(delay)
    browser.save_screenshot('map.png')
    browser.quit()
    
    # crop and save
    from PIL import Image
    image = Image.open(path + '/map.png')
    box = (0, 0, 270, 480)
    cropped_image = image.crop(box)
    cropped_image.save('cropped/{}.png'.format(idx))

KeyboardInterrupt: 

In [143]:
# Create Gif and remove each .png file
from pathlib import Path
import imageio

image_path = Path('C:/Users/cs317917/Documents/GitHub/evolutionary_traveling_salesman/cropped')

images = list(image_path.glob('*.png'))
image_list = []
for file_name in images:
    image_list.append(imageio.imread(file_name))
    #os.remove(file_name)
    
imageio.mimwrite('GifMap.gif', image_list, fps=2)